In [10]:
from sklearn.metrics import adjusted_mutual_info_score as AMI 
import hypernetx as hnx
import hypernetx.algorithms.hypergraph_modularity as hmod
from collections import Counter
import igraph as ig
import pandas as pd
import numpy as np
import h_louvain as hl
import csv
import os
import random
import matplotlib.pyplot as plt
import h_louvain_BO as hlbo
import pickle


In [11]:
import warnings
warnings.filterwarnings("ignore")
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))


In [12]:
def build_habcd():
    cmd = julia + ' -n ' + str(nodes)
    cmd = cmd + ' -d ' + str(deg_exp) + ',' + str(deg_min) + ',' + str(deg_max)
    cmd = cmd + ' -c ' + str(com_exp) + ',' + str(com_min) + ',' + str(com_max)
    cmd = cmd + ' -x ' + str(xi)
    cmd = cmd + ' -q ' + str(edge_sizes[0])
    for i in np.arange(1,len(edge_sizes),1):
        cmd = cmd + ',' + str(edge_sizes[i])
    cmd = cmd + ' -w wcd.txt'
    cmd = cmd + ' -s ' + str(seed)
    cmd = cmd + ' -o hgwcdM '     
    return cmd

In [13]:
def load_ABCDH_from_file(filename):
    with open(filename,"r") as f:
        rd = csv.reader(f)
        lines = list(rd)
    Edges = []
    for line in lines:
        Edges.append(set(line))

    HG = hnx.Hypergraph(dict(enumerate(Edges)))
    return HG

In [14]:
def get_AMI(HG,gt,A):
    d = hmod.part2dict(A)
    A4ami = [d[str(i+1)] for i in range(len(HG.nodes))]
    return AMI(gt, A4ami)


In [15]:
julia = '/work/home/fcthebe/julia-1.7.2/bin/julia --project /work/home/fcthebe/ABCDHypergraphGenerator.jl-main/utils/abcdh.jl'

nodes = 300
deg_min = 5
deg_max = 30
deg_exp = 2.5
com_min = 40
com_max = 60
com_exp = 1.5
edge_sizes = [0,0,0,0,1]
#edge_type = 'majority'

# ## FCT - experimental modularity function


In [16]:
## different graph
xi = 0.2
L = []
def expo(d, c):
    return 1 if c==d else 0  

for rep in range(30):
    seed = rep
    random.seed(rep)

    cmd = build_habcd()
    os.system(cmd)

    with open("./hgwcdM_he.txt","r") as f:
        rd = csv.reader(f)
        lines = list(rd)
    Edges = []
    for line in lines:
        Edges.append(set(line))
    
    with open("./hgwcdM_assign.txt", 'r') as file:
        gt = [int(line) for line in file]
    gt_dict = {str(i+1):gt[i] for i in range(len(gt))}

    mc = max(gt)
    Nodes = [x for x in gt_dict.keys() if gt_dict[x]>=(mc-1)]
    for i in range(35):
        Edges.append(set(np.random.choice(Nodes,5,replace=False)))

    HG = hnx.Hypergraph(dict(enumerate(Edges)))
    A_gt = [x for x in hmod.dict2part(gt_dict) if len(x)>0]
    G = hnx.two_section(HG)
    dct = hnx.part2dict(A_gt)
    for v in G.vs:
        v['gt'] = dct[v['name']]   

    hL = hl.hLouvain(HG, hmod_type=expo, delta_it = 0.0001, delta_phase = 0.0001, random_seed = seed) 
    qH_gt = hL.combined_modularity(A_gt, expo, 1)

    ## two-section
    qH_ts = -1
    for ts_rep in range(5):
        ml = G.community_multilevel(weights='weight').membership
        q = hL.combined_modularity(hnx.dict2part(dict(zip(G.vs['name'], ml))), expo, 0)
        if q > qH_ts:
            ami_ts = AMI(G.vs['gt'],ml)
            qH_ts = q 
    V = [ami_ts]
    ## strict
    hBO = hlbo.hLouvainBO(HG, hmod_type=expo, delta_it=0.0001, delta_phase=0.0001, random_seed = seed)
    hBO.set_params(hmod_type=expo, show_bo_table=False,seeds=[rep,rep+100,rep+200, rep+300, rep+400])
    df_bo = hBO.hLouvain_perform_BO()
    qH_bo = hL.combined_modularity(df_bo["A"].tolist()[0], expo, 1)
    ami_bo = get_AMI(HG, gt, df_bo["A"].tolist()[0])
    V.append(ami_bo)
    V.append(qH_bo)

    ## others
    for ex in [0,1,2,3,4,5,6,7,8]:
        def expo(d, c):
            return (c/d)**(ex/2) if c > d/2 else 0  
        hBO = hlbo.hLouvainBO(HG, hmod_type=expo, delta_it=0.0001, delta_phase=0.0001, random_seed = seed)
        hBO.set_params(hmod_type=expo, show_bo_table=False,seeds=[rep,rep+100,rep+200, rep+300, rep+400])
        df_bo = hBO.hLouvain_perform_BO()
        qH_bo = hL.combined_modularity(df_bo["A"].tolist()[0], expo, 1)
        ami_bo = get_AMI(HG, gt, df_bo["A"].tolist()[0])
        V.append(ami_bo)
        V.append(qH_bo)
    L.append(V)
    print(V)
    fn = 'test_more_5_different.pkl'
    with open(fn, 'wb') as fp:
        pickle.dump(L, fp)


[ Info: distribution of hyperedge composition for d=5 does not add up to 1. Fixing.
[ Info: Moving 4 stumps from community 1 to background graph
[ Info: Moving 3 stumps from community 2 to background graph
[ Info: Moving 2 stumps from community 3 to background graph
[ Info: Moving 4 stumps from community 6 to background graph
[ Info: Moving 3 stumps from community 7 to background graph
[ Info: added degree to the following nodes due to parity issues: [80, 167] 
[ Info: fixing 44 hyperedges (8.43% of total number of hyperedges) that were multisets


[0.773574862166824, 0.38421487816268785, 0.30844422071704036, 0.7297661986597271, 0.6029650641344184, 0.7450454856306894, 0.5162200934052323, 0.8183482102321998, 0.44234562181378817, 0.7361317771323825, 0.3698094545036535, 0.7772330575576, 0.34293309333631233, 0.6764261520351951, 0.32018575406794214, 0.6210850128241908, 0.3054526780555938, 0.5067780100516893, 0.3003058061071731, 0.5178428485050137, 0.29851769319925014]


[ Info: distribution of hyperedge composition for d=5 does not add up to 1. Fixing.
[ Info: Moving 4 stumps from community 3 to background graph
[ Info: Moving 4 stumps from community 4 to background graph
[ Info: Moving 1 stumps from community 5 to background graph
[ Info: Moving 4 stumps from community 6 to background graph
[ Info: added degree to the following nodes due to parity issues: [8] 
[ Info: fixing 49 hyperedges (8.88% of total number of hyperedges) that were multisets


[0.7662399901631728, 0.47120681897635996, 0.27956654162542366, 0.6093586915831746, 0.6190003598521235, 0.6704533828742988, 0.5175130665680638, 0.8042463624479902, 0.4398076173930851, 0.8543416007254601, 0.38189211873916146, 0.8620861558745947, 0.33556285601821456, 0.6697672248700279, 0.30177711656721184, 0.6256675295740741, 0.29264107757923397, 0.6516611761056033, 0.28272581465021035, 0.47120681897635996, 0.27956654162542366]


[ Info: distribution of hyperedge composition for d=5 does not add up to 1. Fixing.
[ Info: Moving 3 stumps from community 1 to background graph
[ Info: Moving 1 stumps from community 2 to background graph
[ Info: Moving 3 stumps from community 3 to background graph
[ Info: Moving 4 stumps from community 4 to background graph
[ Info: Moving 3 stumps from community 5 to background graph
[ Info: Moving 1 stumps from community 6 to background graph
[ Info: added degree to the following nodes due to parity issues: [1] 
[ Info: fixing 48 hyperedges (9.14% of total number of hyperedges) that were multisets


...

[0.8175078241162373, 0.6013967635180585, 0.29549202734959307, 0.4684459499667728, 0.6301496697405926, 0.603963106402056, 0.5120084512299544, 0.7344755172028308, 0.4444265985844277, 0.815057215076897, 0.3866330762743128, 0.828550112294846, 0.344381196113933, 0.8365881948812394, 0.31973340311777915, 0.7174355278180577, 0.3060224482629183, 0.6537638041698256, 0.2987423166839109, 0.6013967635180585, 0.29549202734959307]


[ Info: distribution of hyperedge composition for d=5 does not add up to 1. Fixing.
[ Info: Moving 1 stumps from community 2 to background graph
[ Info: Moving 3 stumps from community 3 to background graph
[ Info: Moving 4 stumps from community 4 to background graph
[ Info: Moving 3 stumps from community 5 to background graph
[ Info: Moving 1 stumps from community 6 to background graph
[ Info: added degree to the following nodes due to parity issues: [77, 204, 113] 
[ Info: fixing 50 hyperedges (9.07% of total number of hyperedges) that were multisets


[0.8161443116973337, 0.5252673685717995, 0.284113929025373, 0.5570340210089768, 0.6036295140401026, 0.7342722435936251, 0.49473689310919267, 0.7858321086609426, 0.4335253886743907, 0.7642383193787127, 0.37558297617549957, 0.8141829801789673, 0.33422511391853915, 0.8125738249969052, 0.2995417114335181, 0.6037505006321178, 0.28573464272176075, 0.5856259646241284, 0.28264442510849747, 0.5252673685717995, 0.284113929025373]


[ Info: distribution of hyperedge composition for d=5 does not add up to 1. Fixing.
[ Info: Moving 3 stumps from community 1 to background graph
[ Info: Moving 1 stumps from community 2 to background graph
[ Info: Moving 3 stumps from community 3 to background graph
[ Info: Moving 4 stumps from community 4 to background graph
[ Info: Moving 2 stumps from community 5 to background graph
[ Info: Moving 3 stumps from community 6 to background graph
[ Info: fixing 53 hyperedges (10.29% of total number of hyperedges) that were multisets


[0.7096308275453296, 0.4941352736948158, 0.28514927482426516, 0.36879598407328557, 0.5785387374067079, 0.7510044729274634, 0.4771213105894276, 0.7211504432196294, 0.42908726292317456, 0.6896567630575574, 0.367893443825126, 0.7296159484147735, 0.3280260540588742, 0.7299777610528041, 0.3064521809927188, 0.6533696235043666, 0.29194133684046814, 0.6252149642906492, 0.2872664766477232, 0.4941352736948158, 0.28514927482426516]
